# Protein Structure Hallucination with AlphaFold2
## Hallucinating a Pembrolizumab-Like Fab Structure

----------------------

### Install ColabDesign for AlphaFold2 and Import System Libraries

In [ ]:
#@title install
%%bash
if [ ! -d af_backprop ]; then
  git clone https://github.com/sokrypton/af_backprop.git
  pip -q install biopython dm-haiku==0.0.5 ml-collections py3Dmol
fi
if [ ! -d params ]; then
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar | tar x -C params
fi
wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/colabfold.py
wget -qnc https://raw.githubusercontent.com/sokrypton/ColabDesign/main/af/design.py

In [ ]:
#@title import libraries
import sys
sys.path.append('/content/af_backprop')

import os
from google.colab import files
import numpy as np
from IPython.display import HTML
from design import mk_design_model, clear_mem

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

## Hallucinate Light Chain

In [ ]:
clear_mem()
model = mk_design_model(protocol="fixbb")
model.prep_inputs(pdb_filename=get_pdb("5JXE"), chain="C")

print("length",  model._len)
print("weights", model.opt["weights"])

In [ ]:
model.design_3stage()

In [ ]:
model.save_pdb(f"{model.protocol}_c_light.pdb")

## Hallucinate Heavy Chain

In [ ]:
clear_mem()
model = mk_design_model(protocol="fixbb")
model.prep_inputs(pdb_filename=get_pdb("5JXE"), chain="D")

print("length",  model._len)
print("weights", model.opt["weights"])

In [ ]:
model.design_3stage()

In [ ]:
model.save_pdb(f"{model.protocol}_d_heavy.pdb")